This notebook compares the high pressure limit rate coefficients in this work with those in Sarathy et al. (citation in paper). 

In [ ]:
import numpy as np
import pandas as pd
import os
import cantera as ct

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_context('notebook')

In [ ]:
mit_red = (163/255,31/255,52/255)
mit_grey = (138/255,139/255,140/255)
mit_light_grey = (194/255,192/255,191/255)

# Sarathy Comparison

In [ ]:
my_solution = ct.Solution('../data/mech_generation/high_p_merged/chem.cti')
sarathy_solution = ct.Solution('../data/lit_mechanisms/sarathy.cti')

In [ ]:
species_conversion_df = pd.read_csv('../data/species_name_comparison.csv')
cantera2sarathy = pd.Series(index=species_conversion_df['chemkin_name'],data=species_conversion_df['sarathy'].values)

In [ ]:
reaction = my_solution.reactions()[0]
sarathy_r_names = set([cantera2sarathy[name] for name in reaction.reactants.keys()])
s_reactants = set(sarathy_solution.reactions()[2122].reactants.keys())

In [ ]:
my_reactions = my_solution.reactions()
sarathy_reactions = sarathy_solution.reactions()

In [ ]:
# get dictionary linking my reactions with sarathy reactions
my_index2sarathy = {}
for my_index, reaction in enumerate(my_reactions):
    sarathy_r_names = set([cantera2sarathy[name] for name in reaction.reactants.keys()])
    sarathy_p_names = set([cantera2sarathy[name] for name in reaction.products.keys()])
    for sarathy_index, s_reaction in enumerate(sarathy_reactions):
        s_reactants = set(s_reaction.reactants.keys())
        s_products  = set(s_reaction.products.keys())
        #print(str(my_index) +'\t' + str(sarathy_index)+'\t'+str(s_reactants) + '\t\t\t\t' + str(sarathy_r_names))
        if s_reactants == sarathy_r_names or s_reactants == sarathy_p_names:
            if s_reactants == sarathy_r_names:
                forward = True
            elif s_reactants == sarathy_p_names:
                forward = False
            else:
                raise Exception('Something went wrong in reaction index {0} sarathy index {1}'.format(my_index,sarathy_index))
            if forward and s_products == sarathy_p_names:
                my_index2sarathy[my_index] = sarathy_index
                break
            elif (not forward) and s_products == sarathy_r_names:
                my_index2sarathy[my_index] = -sarathy_index
                break


In [ ]:
# add waddington reaction:
my_reactions[26]

In [ ]:
sarathy_reactions[2091]

In [ ]:
my_index2sarathy[26] = 2091

In [ ]:
# spaced temperatures by inverse spacing between two set values below
t_range = 1/np.linspace(1./500,1./1000)

In [ ]:
# get data for all conditions (so we don't have to recalculate it each time)
my_rates = pd.DataFrame(index = range(len(my_reactions)), columns=t_range)
sarathy_rates = pd.DataFrame(index= range(len(sarathy_reactions)), columns=t_range)
sarathy_rates_reverse = pd.DataFrame(index= range(len(sarathy_reactions)), columns=t_range)
for t in t_range:
    my_solution.TP = t, 1e5
    sarathy_solution.TP = t, 1e5
    my_rates[t] = my_solution.forward_rate_constants
    sarathy_rates[t] = sarathy_solution.forward_rate_constants
    sarathy_rates_reverse[t] = sarathy_solution.reverse_rate_constants

In [ ]:
rate_comparison = pd.Series()
for my_i, sarathy_i in my_index2sarathy.items():
    if sarathy_i < 0:
        sarathy_i *= -1
        sarathy_data = sarathy_rates_reverse.loc[sarathy_i,:]
    else:
        sarathy_data = sarathy_rates.loc[sarathy_i,:]
    my_data = my_rates.loc[my_i,:]
    rate_comparison[str(my_reactions[my_i])] = (my_data/sarathy_data).mean()

In [ ]:
decomp_comparison = pd.Series()
QOOH_decomp_comparison = pd.Series()
QOOH_decomp_minus_HO2 = pd.Series()
epoxy_comp = pd.Series()
c4_comp = pd.Series()
# remove the O2 rates & make new series with only decomposition rates
for label in rate_comparison.index:
    if str.startswith(label,'O2 '):
        del rate_comparison[label]
    elif '+' in label or label == 'bRO2 <=> bQOOHa':
        # set rxn as decomposition if it has a + or is waddington rxn (special case in sarathy mech)
        decomp_comparison[label] = rate_comparison[label]
        if 'RO2' not in label:
            QOOH_decomp_comparison[label] = rate_comparison[label]
            if 'HO2' not in label:
                QOOH_decomp_minus_HO2[label] = rate_comparison[label]
                if 'epoxy' in label:
                    epoxy_comp[label] = rate_comparison[label]
                elif 'c4ether' in label:
                    c4_comp[label] = rate_comparison[label]

In [ ]:
f, ax = plt.subplots(figsize=(8, 4))

plt.hist(np.log10(rate_comparison.values),bins=[-3,-2,-1,0,1,2,3,4,5,6,7],color=mit_red)
plt.hist(np.log10(decomp_comparison.values),bins=[-3,-2,-1,0,1,2,3,4,5,6,7],color=mit_grey)
plt.hist(np.log10(QOOH_decomp_comparison.values),bins=[-3,-2,-1,0,1,2,3,4,5,6,7],color=mit_light_grey, hatch='/')
plt.hist(np.log10(QOOH_decomp_minus_HO2.values),bins=[-3,-2,-1,0,1,2,3,4,5,6,7],color=mit_light_grey, hatch='*')
plt.hist(np.log10(epoxy_comp.values),bins=[-3,-2,-1,0,1,2,3,4,5,6,7],color=mit_light_grey, hatch='+')
plt.hist(np.log10(c4_comp.values),bins=[-3,-2,-1,0,1,2,3,4,5,6,7],color=mit_light_grey, hatch='o')

sns.despine()
ax.set_xlabel('ratio of reaction rates in this work to those in a combustion mechanism')
ax.set_ylabel("number of reactions")
ax.set_xticks([-2,0,2,4,6])
ax.set_xticklabels(['$10^{'+ str(i) +'}$' for i in ax.get_xticks()])

ax.annotate('C$_4$ Cyclic Ether\nFormation',(-.8,.85), (-3.35,2), arrowprops = {'arrowstyle': '-'})
ax.annotate('Epoxy Formation',(6.5,1.7), (4.9,3.1), arrowprops = {'arrowstyle': '-'})
ax.annotate('HO$_2$ from RO$_2$',(.2,3), (-3.1,4.5), arrowprops = {'arrowstyle': '-'})
ax.annotate('Isomerization',(.2,5.5), (-2.85,5.9), arrowprops = {'arrowstyle': '-'})
ax.annotate('HO$_2$ from QOOH',(1.8,4), (3,6), arrowprops = {'arrowstyle': '-'})
ax.annotate(u'QOOH β-scission',(1.8,2.4), (2.9,4), arrowprops = {'arrowstyle': '-'})


In [ ]:
# matplotlib bug: To save properly as pdf with hatches, save as svg, open inkscape, and save to pdf
f.savefig(os.path.join('../results/','sarathy_rate_comparison.svg'),bbox_inches='tight')

In [ ]:
f.savefig(os.path.join('../results/','sarathy_rate_comparison.jpg'),dpi=450,bbox_inches='tight')